### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [1]:
import sys  
sys.path.insert(0, '../../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 

import operator

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

2023-07-24 16:53:53.683345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 16:53:54.671004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Model creation function

In [2]:
def create_model(learning_rate=0.0001, input_dim=10, number_layers=1, neurons_per_layer=20):
    # create model
    model = Sequential()

    model.add(Dense(neurons_per_layer, input_shape=(input_dim,), activation='relu'))
    for _ in range(number_layers):
        model.add(Dense(neurons_per_layer, activation='relu'))
    model.add(Dense(1, activation=None))

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model


### Hyperparameter space

In [3]:
hyperparameterspace = {
    'epochs': ["interval-int", 1, 20],
    #'batch_size': ["interval-int", 1, 200],
    'learning rate': ["interval-log", 1e-10, 1e-1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

### Optimization

In [4]:
def visualize(points, budget, alternative):

    points.sort(key=operator.attrgetter('value'))
    x_values = []
    y_values = []
    z_values = []

    print(len(points))

    for i in range(len(points)):    
        x_values.append(points[i].get_coordinates()[0])
        y_values.append(np.log10(points[i].get_coordinates()[1]))
        z_values.append(points[i].get_value())
        print(i)
        print(points[i].get_value())
        print(points[i].get_coordinates())


    fig = plt.figure()
    ax = plt.axes()
    surface = plt.scatter(x_values, y_values, c=z_values, cmap="plasma")
    plt.scatter(points[0].get_coordinates()[0], points[0].get_coordinates()[1], c="white", marker="x")

    plt.colorbar(surface)
    # plt.gca().set_aspect("equal")

    plt.xlabel("Epochs")
    plt.ylabel("Log of learning rate")

    plt.savefig("./Alternative_"+str(alternative)+"_budget_"+str(budget)+".pgf",bbox_inches='tight' )

    plt.show()

In [5]:

################## MODEL AND FUNCTION DEFINITION ####################
CV = 2
dataset = HPO.Dataset(task_id=233211)

def evaluate_model(epochs, batch_size, learning_rate, number_of_layers, neurons_per_layer, deterministic=True):

    kfold = KFold(n_splits=CV)

    split = (kfold.split(dataset.get_X(), dataset.get_Y()))

    values = []

    numeric_features = [not x for x in dataset.get_categorical_indicator()]
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("encoder", OneHotEncoder(
                handle_unknown="infrequent_if_exist", sparse_output=False)),
            # ("selector", SelectPercentile(chi2, percentile=50)),
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer,
                dataset.get_categorical_indicator()),
        ]
    )

    for i, (train_index, test_index) in enumerate(split):

        if deterministic:
            reset_seeds()

        X_train = dataset.get_X()[train_index]
        Y_train = dataset.get_Y()[train_index]

        X_val = dataset.get_X()[test_index]
        Y_val = dataset.get_Y()[test_index]

        preprocessor.fit(X_train, Y_train)

        X_train = preprocessor.transform(X_train)
        X_val = preprocessor.transform(X_val)

        regressor = KerasRegressor(model=create_model,
                                    learning_rate=learning_rate,
                                    input_dim=len(
                                        X_train[0]),
                                    number_layers=number_of_layers,
                                    neurons_per_layer=neurons_per_layer,
                                    verbose=0)

        regressor = TransformedTargetRegressor(regressor=regressor,
                                                transformer=StandardScaler())

        regressor.fit(X_train, Y_train, epochs=int(epochs),
                        batch_size=int(batch_size), shuffle=False)

        Y_predicted = regressor.predict(X_val)
        # error = sklearn.metrics.mean_absolute_error(Y_predicted, Y_val)
        error = sklearn.metrics.mean_absolute_percentage_error(
            Y_predicted, Y_val)
        values.append(error)

        del regressor
        K.clear_session()

    result = sum(values)/len(values)
    return result

def function(coordinates):
        
    return evaluate_model(epochs=coordinates[0], batch_size=400, learning_rate=coordinates[1], number_of_layers=1, neurons_per_layer=30, deterministic=False)


BUDGETS = []

for budget in BUDGETS:

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.75, 
                                                        init_points=10*len(hyperparameterspace.keys()), 
                                                        alternative=0, ref_per_step=4)
    points = optimizer.fit()

    visualize(points, budget, 0)

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.35, 
                                                        init_points=8*len(hyperparameterspace.keys()), 
                                                        alternative=1, ref_per_step=4)
    points = optimizer.fit()

    visualize(points, budget, 1)

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.6, 
                                                        init_points=7*len(hyperparameterspace.keys()), 
                                                        alternative=2, ref_per_step=4)
    points = optimizer.fit()

    visualize(points, budget, 2)


In [6]:
def blackboxfunction_grid(params):
    # index = int(params[0]*(len(hyperparameterspace_special["loss"])-1))
    # hyperparameterspace_special["loss"][index]

    epochs = int(params[0])

    batch_size = 400

    learning_rate = params[1]

    number_of_layers = 1

    neurons_per_layer = 30

    return evaluate_model(epochs, batch_size, learning_rate, number_of_layers, neurons_per_layer)

def blackboxfunction_random(params):
    # index = int(params[0]*(len(hyperparameterspace_special["loss"])-1))
    # hyperparameterspace_special["loss"][index]

    epochs = int(params[0])

    batch_size = 400

    learning_rate = params[1]

    number_of_layers = 1

    neurons_per_layer = 30

    return evaluate_model(epochs, batch_size, learning_rate, number_of_layers, neurons_per_layer, deterministic=False)

def blackboxfunction_bayesian(params):
    # index = int(params[0]*(len(hyperparameterspace_special["loss"])-1))
    # hyperparameterspace_special["loss"][index]

    epochs = int(params[0])

    batch_size = 400

    model_learning_rate = 10 ** (params[1])

    number_of_layers = 1

    neurons_per_layer = 30

    return evaluate_model(epochs, batch_size, model_learning_rate, number_of_layers, neurons_per_layer, deterministic=False)

##################### Function for sparse grid search #####################

class ExampleFunction(pysgpp.ScalarFunction):

    def __init__(self):
        super(ExampleFunction, self).__init__(
            len(hyperparameterspace.keys()))

    def eval(self, x):
        # index = int(x[0]*(len(hyperparameterspace_special["loss"])-1))
        # hyperparameterspace_special["loss"][index]

        epochs = int(HPO.from_standard(
            hyperparameterspace_special["epochs"][0], hyperparameterspace_special["epochs"][1], x[0]))

        batch_size = 400 # int(HPO.from_standard(
           #  hyperparameterspace_special["batch_size"][0], hyperparameterspace_special["batch_size"][1], x[1]))

        # HPO.from_standard_log(hyperparameterspace_special["learning_rate"][
        model_learning_rate = HPO.from_standard_log(
            hyperparameterspace_special["learning rate"][0], hyperparameterspace_special["learning rate"][1], x[1])

        number_of_layers = 1 #  int(HPO.from_standard(
            # hyperparameterspace_special["number_layers"][0], hyperparameterspace_special["number_layers"][1], x[3]))

        neurons_per_layer = 30 #int(HPO.from_standard(
            # hyperparameterspace_special["neurons_per_layer"][0], hyperparameterspace_special["neurons_per_layer"][1], x[4]))
    

        return evaluate_model(epochs, batch_size, model_learning_rate, number_of_layers, neurons_per_layer)


ALT_1 = "{"
ALT_2 = "{"
ALT_3 = "{"

GRID = "{"
RANDOM = "{"
BAYESIAN = "{"
SPARSE = "{"

BUDGETS = [12, 20, 30, 40, 50]

for budget in BUDGETS:

    reset_seeds()


    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.65, 
                                                        init_points=8, 
                                                        alternative=0, ref_per_step=2)
    points = optimizer.fit()

    print(points[0].get_value())
    
    ALT_1 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"
    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.55, 
                                                        init_points=8, 
                                                        alternative=1, ref_per_step=2)
    points = optimizer.fit()
    print(points[0].get_value())

    ALT_2 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"
    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.6, 
                                                        init_points=8, 
                                                        alternative=2, ref_per_step=2)
    points = optimizer.fit()
    print(points[0].get_value())

    ALT_3 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"

    reset_seeds()

    print("\nPerforming grid search")
    optimization = HPO.GridSearchOptimization(
        dataset, blackboxfunction_grid, hyperparameterspace, budget=budget, verbosity=VERBOSE, cv=CV)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    print("Best score with Grid search:", best_score)


    GRID += "(" + str(cost) + "," + str(best_score) + ")"

    print(str(cost) + "," + str(best_score))

    K.clear_session()

    # ########################### RANDOM SEARCH #######################
    print("\nPerforming random search")
    reset_seeds()

    optimization = HPO.RandomSearchOptimization(
        dataset, blackboxfunction_random, hyperparameterspace, budget=budget, verbosity=VERBOSE, cv=CV)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    RANDOM += "(" + str(cost) + "," + str(best_score) + ")"
    print("Best score with Random search:", best_score)

    print(str(cost) + "," + str(best_score))


    K.clear_session()
    reset_seeds()

    ########################### BAYESIAN OPT #####################
    print("\nPerforming bayesian optimization")

    optimization = HPO.BayesianOptimization(
        dataset, blackboxfunction_bayesian, hyperparameterspace, budget=budget, verbosity=VERBOSE)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    BAYESIAN += "(" + str(budget) + "," + str(best_score) + ")"
    print("Best score with Bayesian Optimization:", best_score)


    print(str(budget) + "," + str(best_score))


    K.clear_session()

    ########################### SPARSE OPT ############################
    reset_seeds()

    print("\nPerforming sparse search")

    f = ExampleFunction()

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=budget, verbosity=VERBOSE, degree=2, adaptivity=0.85, optimizer="gradient_descent")

    [X0, fX0, X1, fX1, X2, fX2], cost = optimization.fit()

    cost = cost + 2
    bestFX = fX0 
    if fX1 < bestFX:
        bestFX = fX1 
    if fX2 < bestFX:
        bestFX = fX2


    SPARSE += "(" + str(cost) + "," + str(bestFX) + ")"

    print("Sparse grid search")
    print(str(cost) + "," + str(bestFX))
        
    K.clear_session()

    print(ALT_1)
    print(ALT_2)
    print(ALT_3)
    print(GRID)
    print(RANDOM)
    print(BAYESIAN)  
    print(SPARSE)  

2023-07-24 16:53:57.228281: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


0.561686709523201
0.6530265212059021
0.5859223902225494

Performing grid search
Best score with Grid search: 0.5451523661613464
9,0.5451523661613464

Performing random search
Best score with Random search: 0.4797375500202179
12,0.4797375500202179

Performing bayesian optimization


/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Best score with Bayesian Optimization: 0.2904946506023407
12,0.2904946506023407

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 45858ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
11,0.557055801153183
{(12,0.561686709523201)
{(12,0.6530265212059021)
{(12,0.5859223902225494)
{(9,0.5451523661613464)
{(12,0.4797375500202179)
{(12,0.2904946506023407)
{(11,0.557055801153183)
0.561686709523201
0.6388656198978424
0.5859223902225494

Performing grid search
Best score with Grid search: 0.4490787833929062
16,0.4490787833929062

Performing random search
Best score with Random search: 0.4797375500202179
20,0.4797375500202179

Performing bayesian optimization


/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


...

Best score with Bayesian Optimization: 0.2904946506023407
30,0.2904946506023407

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 129063ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
27,0.31314781308174133
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)
{(12,0.4797375500202179)(20,0.4797375500202179)(30,0.4797375500202179)
{(12,0.2904946506023407)(20,0.2904946506023407)(30,0.2904946506023407)
{(11,0.557055801153183)(19,0.38420042395591736)(27,0.31314781308174133)
0.3593377470970154
0.4123602956533432
0.4224883168935776

Performing grid search
Best score with Grid search: 0.5727062821388245
36,0.5727062821388245



/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2904946506023407
40,0.2904946506023407

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 189376ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
39,0.2810547798871994
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)(36,0.5727062821388245)
{(12,0.4797375500202179)(20,0.4797375500202179)(30,0.4797375500202179)(40,0.3352629020810127)
{(12,0.2904946506023407)(20,0.2904946506023407)(30,0.2904946506023407)(40,0.2904946506023407)
{(11,0.557055801153183)(19,0.38420042395591736)(27,0.31314781308174133)(39,0.2810547798

/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2672734633088112
50,0.2672734633088112

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 227162ms.
Solving linear system (automatic method)...
Done in 1ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
47,0.2810547798871994
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)(50,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)(50,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)(50,0.2970848008990288)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)(36,0.5727062821388245)(49,0.40663371980190277)
{(12,0.4797375500202179)(20,0.4797375500202179)(30,0.4797375500202179)(40,0.3352629020810127)(50,0.3352629020810127)
{(12,0.2904946506023407)(20,0.2904946506023407)(30,0.290494650602

In [7]:

BUDGETS = [60, 70, 80, 90]

for budget in BUDGETS:

    reset_seeds()


    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.65, 
                                                        init_points=8, 
                                                        alternative=0, ref_per_step=2)
    points = optimizer.fit()

    print(points[0].get_value())
    
    ALT_1 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"
    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.55, 
                                                        init_points=8, 
                                                        alternative=1, ref_per_step=2)
    points = optimizer.fit()
    print(points[0].get_value())

    ALT_2 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"
    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, 
                                                        hyperparameterspace, budget, 0, 0.6, 
                                                        init_points=8, 
                                                        alternative=2, ref_per_step=2)
    points = optimizer.fit()
    print(points[0].get_value())

    ALT_3 += "(" + str(budget) + "," + str(points[0].get_value()) + ")"

    reset_seeds()

    print("\nPerforming grid search")
    optimization = HPO.GridSearchOptimization(
        dataset, blackboxfunction_grid, hyperparameterspace, budget=budget, verbosity=VERBOSE, cv=CV)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    print("Best score with Grid search:", best_score)


    GRID += "(" + str(cost) + "," + str(best_score) + ")"

    print(str(cost) + "," + str(best_score))

    K.clear_session()

    # ########################### RANDOM SEARCH #######################
    print("\nPerforming random search")
    reset_seeds()

    optimization = HPO.RandomSearchOptimization(
        dataset, blackboxfunction_random, hyperparameterspace, budget=budget, verbosity=VERBOSE, cv=CV)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    RANDOM += "(" + str(cost) + "," + str(best_score) + ")"
    print("Best score with Random search:", best_score)

    print(str(cost) + "," + str(best_score))


    K.clear_session()
    reset_seeds()

    ########################### BAYESIAN OPT #####################
    print("\nPerforming bayesian optimization")

    optimization = HPO.BayesianOptimization(
        dataset, blackboxfunction_bayesian, hyperparameterspace, budget=budget, verbosity=VERBOSE)

    result, cost = optimization.fit()

    index_best = 0
    for m in range(len(result)):
        if result[m][1] < result[index_best][1]:
            index_best = m

    best_score = result[index_best][1]
    best_params = result[index_best][0]


    BAYESIAN += "(" + str(budget) + "," + str(best_score) + ")"
    print("Best score with Bayesian Optimization:", best_score)


    print(str(budget) + "," + str(best_score))


    K.clear_session()

    ########################### SPARSE OPT ############################
    reset_seeds()

    print("\nPerforming sparse search")

    f = ExampleFunction()

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=budget, verbosity=VERBOSE, degree=2, adaptivity=0.85, optimizer="gradient_descent")

    [X0, fX0, X1, fX1, X2, fX2], cost = optimization.fit()

    cost = cost + 2
    bestFX = fX0 
    if fX1 < bestFX:
        bestFX = fX1 
    if fX2 < bestFX:
        bestFX = fX2


    SPARSE += "(" + str(cost) + "," + str(bestFX) + ")"

    print("Sparse grid search")
    print(str(cost) + "," + str(bestFX))
        
    K.clear_session()

    print(ALT_1)
    print(ALT_2)
    print(ALT_3)
    print(GRID)
    print(RANDOM)
    print(BAYESIAN)  
    print(SPARSE)  

0.3593377470970154
0.4123602956533432
0.2970848008990288

Performing grid search
Best score with Grid search: 0.40663371980190277
49,0.40663371980190277

Performing random search
Best score with Random search: 0.33082716166973114
60,0.33082716166973114

Performing bayesian optimization


/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2672734633088112
60,0.2672734633088112

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 301804ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
59,0.2552933469414711
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)(50,0.3593377470970154)(60,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)(50,0.4123602956533432)(60,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)(50,0.2970848008990288)(60,0.2970848008990288)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)(36,0.5727062821388245)(49,0.40663371980190277)(49,0.40663371980190277)
{(12,0.4797375500202179)(20,0.4797375500202179)(30,0.4797375500202179)(40,0.3352629020810

/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2672734633088112
70,0.2672734633088112

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 356589ms.
Solving linear system (automatic method)...
Done in 1ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 3ms.
Sparse grid search
67,0.2552933469414711
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)(50,0.3593377470970154)(60,0.3593377470970154)(70,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)(50,0.4123602956533432)(60,0.4123602956533432)(70,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)(50,0.2970848008990288)(60,0.2970848008990288)(70,0.2970848008990288)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)(36,0.5727062821388245)(49,0.40663371980190277)(49,0.40663371980190277)(64,0.325645044445991

/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2672734633088112
80,0.2672734633088112

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 421862ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
79,0.2552933469414711
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)(50,0.3593377470970154)(60,0.3593377470970154)(70,0.3593377470970154)(80,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)(50,0.4123602956533432)(60,0.4123602956533432)(70,0.4123602956533432)(80,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)(50,0.2970848008990288)(60,0.2970848008990288)(70,0.2970848008990288)(80,0.2970848008990288)
{(9,0.5451523661613464)(16,0.4490787833929062)(25,0.3621925860643387)(36,0.5727062821388245)

/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/maxi/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: 

Best score with Bayesian Optimization: 0.2672734633088112
90,0.2672734633088112

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 468710ms.
Solving linear system (automatic method)...
Done in 2ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.
Sparse grid search
87,0.2552933469414711
{(12,0.561686709523201)(20,0.561686709523201)(30,0.3593377470970154)(40,0.3593377470970154)(50,0.3593377470970154)(60,0.3593377470970154)(70,0.3593377470970154)(80,0.3593377470970154)(90,0.3593377470970154)
{(12,0.6530265212059021)(20,0.6388656198978424)(30,0.4123602956533432)(40,0.4123602956533432)(50,0.4123602956533432)(60,0.4123602956533432)(70,0.4123602956533432)(80,0.4123602956533432)(90,0.4123602956533432)
{(12,0.5859223902225494)(20,0.5859223902225494)(30,0.5859223902225494)(40,0.4224883168935776)(50,0.2970848008990288)(60,0.2970848008990288)(70,0.2970848008990288)(80,0.2970848008990288)(90,0.2970848008990288)
{(9,0.5451523661613464)